In [7]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()
print(vocab_size)

1027


In [2]:
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

x = torch.tensor([0, 2])
one_hot(x, vocab_size)




tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [3]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)
print(inputs)

5 torch.Size([2, 1027])
[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]


In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


will use cuda


In [5]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )


In [6]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)



In [7]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape) 


5 torch.Size([2, 1027]) torch.Size([2, 256])


In [8]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        print(X)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [9]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)


[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]


'分开断初邂手承己简野器看'

In [10]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)


In [14]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()

            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))


In [12]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']


In [13]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)


uda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')]
[tensor([[0., 0., 0.,  ..., 0.,

In [26]:
data_iter = d2l.data_iter_random(corpus_indices, batch_size, num_steps, device)

In [27]:
for X, y in data_iter:
    print(X)
    print(X.shape)
    print(type(X))
    inputs = to_onehot(X, 10)
    print(type(inputs))
    print(inputs[0])
    print(inputs[0].shape)
    break

tensor([[ 261.,  261.,  263.,  ..., 1022.,  804.,  152.],
        [ 566., 1009.,  245.,  ...,  631.,  448.,  324.],
        [ 971.,  190.,  575.,  ...,    0.,  324.,  577.],
        ...,
        [ 591.,  454., 1022.,  ...,  448.,  988., 1022.],
        [ 248., 1008., 1022.,  ...,  264., 1022.,  674.],
        [ 423.,  324.,  385.,  ...,  927.,   66.,  168.]], device='cuda:0')
torch.Size([32, 35])
<class 'torch.Tensor'>
<class 'list'>
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0.,

AttributeError: 'list' object has no attribute 'size'

In [106]:
import torch
x = torch.arange(24).reshape(2, 3, 4)
l = nn.LSTM(input_size=4, hidden_size=5, num_layers=2)
state = None
for _ in range(10000):
    y, state = l(x.float(), state)

In [107]:
print(state[0].shape)

torch.Size([2, 3, 5])


In [108]:
print(h)

tensor([[[ 8.8804e-02, -8.5487e-03,  9.2469e-02,  1.9737e-01, -6.2592e-02],
         [ 8.0306e-02, -5.2223e-05,  1.2938e-02,  1.1293e-01, -1.2486e-02],
         [ 4.9383e-02, -2.0724e-07,  1.7562e-03,  5.7833e-02, -2.0174e-03]],

        [[ 9.5629e-02,  4.8861e-02, -8.6619e-02, -2.5372e-02,  2.1580e-01],
         [ 8.8721e-02,  5.2430e-02, -9.0381e-02, -3.7910e-02,  2.2498e-01],
         [ 8.4213e-02,  5.1157e-02, -8.6353e-02, -4.3962e-02,  2.3711e-01]]],
       grad_fn=<StackBackward>)


In [62]:
 y = l(x.float(), (h,c))

In [63]:
y

(tensor([[[ 0.1303, -0.0056,  0.0333, -0.2250,  0.0704],
          [ 0.1171, -0.0193,  0.0474, -0.2487,  0.0881]],
 
         [[ 0.0634, -0.0803,  0.0697, -0.2896,  0.1051],
          [ 0.0673, -0.0893,  0.0781, -0.3115,  0.1183]],
 
         [[ 0.0632, -0.0837,  0.0715, -0.2994,  0.1142],
          [ 0.0706, -0.0890,  0.0781, -0.3177,  0.1271]]],
        grad_fn=<TransposeBackward0>),
 (tensor([[[-1.6319e-02,  1.0577e-01, -5.4034e-01,  5.3460e-02, -6.4752e-02],
           [-4.3859e-06,  6.9505e-02, -7.2742e-01,  5.7375e-03,  1.0671e-01],
           [-3.7784e-10,  1.2939e-02, -7.5732e-01,  7.7466e-04,  3.7851e-02]],
  
          [[ 1.1706e-01, -1.9299e-02,  4.7393e-02, -2.4871e-01,  8.8145e-02],
           [ 6.7296e-02, -8.9316e-02,  7.8069e-02, -3.1147e-01,  1.1830e-01],
           [ 7.0560e-02, -8.8997e-02,  7.8114e-02, -3.1775e-01,  1.2706e-01]]],
         grad_fn=<StackBackward>),
  tensor([[[-1.6858e-02,  1.1098e-01, -8.7103e-01,  5.6414e-02, -6.6475e-02],
           [-4.3867e-06,

In [135]:
  import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def mySample(data):
    num = 0
    total = len(train_set)
    sample_list = []

    for X, y in train_set:
        num = num + y
    

    class_sample_count = np.array([total - num, num])
    weight = 1. / class_sample_count

    for X, y in train_set:
        sample_list.append(weight[int(y)])

    sample_weight = torch.FloatTensor(sample_list)
    sampler = WeightedRandomSampler(sample_weight, total)
    return sampler

class MyData(Dataset):
    def __init__(self, traindata, numstep, length):
        self.serial_number = traindata['serial_number'].value_counts()
        self.value = traindata.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.datalabel = train_data.loc[:, 'label'].values.tolist()
        self.input = []
        self.label = []

        for diskname in self.serial_number.index.sort_values():
            traindata_name = traindata[traindata.serial_number == diskname]
            self.datalabel = traindata_name.loc[:, 'label'].values.tolist()
            for i in range(len(traindata_name) - numstep):
                self.input.append(torch.Tensor(self.datas[i: i + numstep]))
                if(torch.Tensor(self.datalabel[i: i + numstep]).sum() != 0):
                    self.label.append(torch.ones(1))
                else:
                    self.label.append(torch.zeros(1))
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]

train_data = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
train_set = MyData(train_data, 7, 5)
sampler = mySample(train_set)
train_iter = DataLoader(MyData(train_data, 7, 5), batch_size = 10, sampler=sampler, drop_last=True)

In [119]:
conv1 = nn.Conv1d(in_channels = 13, out_channels = 128, kernel_size = 4) # 要求转置
lstmlayer = nn.LSTM(input_size=128, hidden_size=128, num_layers=2)
linear = nn.Linear(128, 2)

In [136]:
state = None
for X, y in train_iter:
    pad = nn.ZeroPad2d(padding=(2, 1, 0, 0))
    X = X.permute(0, 2, 1)
    X = pad(X)
    X = conv1(X)
    X =  X.permute(0, 2, 1) 
    print(X.shape)
    X, state = lstmlayer(X, state)
    X = X[:, X.size(1) - 1, :]
    x = torch.sigmoid(X)
    X = linear(X)
    
    

torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])
torch.Size([10, 7, 128])


In [111]:
X

tensor([[-0.0696, -0.0131],
        [-0.0707, -0.0133],
        [-0.0714, -0.0135],
        [-0.0719, -0.0136],
        [-0.0713, -0.0137],
        [-0.0717, -0.0136],
        [-0.0729, -0.0145]], grad_fn=<AddmmBackward>)

In [112]:
X.shape

torch.Size([7, 2])

In [134]:
from torch.nn.utils.rnn import pack_padded_sequence
for X, y in train_iter:
    print(X.shape)

torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 7, 13])
torch.Size([10, 

In [5]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch.optim import lr_scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def mySample(data):
    num = 0
    total = len(train_set)
    sample_list = []

    for X, y in train_set:
        num = num + y
    

    class_sample_count = np.array([total - num, num])
    weight = 1. / class_sample_count

    for X, y in train_set:
        sample_list.append(weight[int(y)])

    sample_weight = torch.FloatTensor(sample_list)
    sampler = WeightedRandomSampler(sample_weight, total)
    return sampler

class MyData(Dataset):
    def __init__(self, traindata, numstep, length):
        self.serial_number = traindata['serial_number'].value_counts()
        self.value = traindata.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.datalabel = train_data.loc[:, 'label'].values.tolist()
        self.input = []
        self.label = []

        for diskname in self.serial_number.index.sort_values():
            traindata_name = traindata[traindata.serial_number == diskname]
            self.datalabel = traindata_name.loc[:, 'label'].values.tolist()
            for i in range(len(traindata_name) - numstep):
                self.input.append(torch.Tensor(self.datas[i: i + numstep]))
                if(torch.Tensor(self.datalabel[i: i + numstep]).sum() != 0):
                    self.label.append(torch.ones(1))
                else:
                    self.label.append(torch.zeros(1))
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]

train_data = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
train_set = MyData(train_data, 7, 5)
sampler = mySample(train_set)
train_iter = DataLoader(MyData(train_data, 7, 5), batch_size = 10, sampler=sampler, drop_last=True)

In [151]:
p = 0
n = 0
for i in range(10):
    p = 0
    n = 0
    for X, y in train_iter:
        if(y.sum() == 1):
            p = p + 1
        if(y.sum() == 0):
            n = n + 1
    print(n, p)

3659 3628
3674 3613
3602 3685
3602 3685
3625 3662
3566 3721
3702 3585
3728 3559
3651 3636
3567 3720


In [2]:
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, input_seq):
        assert len(input_seq.size()) > 2

        # reshape input data --> (samples * timesteps, input_size)
        # squash timesteps
        reshaped_input = input_seq.contiguous().view(-1, input_seq.size(-1))

        output = self.module(reshaped_input)
        # We have to reshape Y
        if self.batch_first:
            # (samples, timesteps, output_size)
            output = output.contiguous().view(input_seq.size(0), -1, output.size(-1))
        else:
            # (timesteps, samples, output_size)
            output = output.contiguous().view(-1, input_seq.size(1), output.size(-1))
        return output


In [3]:
conv1 = nn.Conv1d(in_channels = 13, out_channels = 128, kernel_size = 4)
Time = TimeDistributed(conv1, True)

In [7]:
for X, y in train_iter:
    print(X.shape)
    output = Time(X)
    print(output.shape)
    break

torch.Size([10, 7, 13])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [128, 13, 4], but got 2-dimensional input of size [70, 13] instead

In [51]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd 

disk = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
data = disk.iloc[:, 5:-1].values
max = np.max(data)
min = np.min(data)
scalar = max - min
data_scalar = list(map(lambda x: x / scalar, data))
targets = disk.loc[:, 'label']
tragets = np.array(targets)

data_gen = TimeseriesGenerator(data_scalar, targets,
                               length=10, sampling_rate=1,
                               batch_size=10)




In [ ]:
def model_fit(train_X, train_y, test_X, test_y):
    # define model
    model = Sequential()
    # model.add(TimeDistributed(cnn))
    model.add(TimeDistributed(Convolution1D(128, 4, border_mode='same'), input_shape=train_X.shape[1:]))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128, return_sequences=True, name="lstm_layer0")) #output = batch * step * output_size
    model.add(LSTM(128, return_sequences=False, name="lstm_layer1")) #output = batch * output_size(last step of every batch)
    # model.add(LSTM(100, return_sequences=True, name="lstm_layer2"))
    model.add(Dense(output_dim, activation='sigmoid'))
    # model.add(GlobalAveragePooling1D(name="global_avg"))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    model.fit(train_X, train_y,batch_size=batch_size, nb_epoch=nb_epoch, verbose=0, validation_data=(test_X, test_y))

    test_y = test_y.reshape(test_y.size, 1)
    predict_y = model.predict(test_X)
    predict_y = predict_y.reshape(predict_y.size, 1)
    predict_y = classifyRes(predict_y)
    calMetrix(__file__, predict_y, test_y)
    t0 = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))

In [56]:
from keras.layers import Dense, LSTM
from keras.layers import Convolution1D, MaxPooling1D, Flatten
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed #输入至少为3D张量
# from matplotlib import pyplot as plt
# import random
import argparse
import time
from keras.preprocessing.sequence import TimeseriesGenerator

text_model = Sequential()
text_model.add(Convolution1D(128, 4, border_mode='same'))
for x, y in data_gen:
    model(x) 

ValueError: Layer sequential_1 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        ...,
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ]],

       [[1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010101, 0.6969697 ],
        [1.01010101, 1.01010101, 1.01010101, ..., 0.85858586,
         1.01010

In [38]:
x, y = data_gen[-1]
y

array([0, 0, 0], dtype=int64)

In [33]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

data = np.array([[i] for i in range(50)])
targets = np.array([[i] for i in range(50)])

data_gen = TimeseriesGenerator(data, targets,
                               length=10, sampling_rate=1,
                               batch_size=2)
assert len(data_gen) == 20

batch_0 = data_gen[0]
x, y = batch_0

for x, y in data_gen:
    print(x)
    print('*************')
    print(y)
    print('****************************************')
    break

[[[ 0]
  [ 1]
  [ 2]
  [ 3]
  [ 4]
  [ 5]
  [ 6]
  [ 7]
  [ 8]
  [ 9]]

 [[ 1]
  [ 2]
  [ 3]
  [ 4]
  [ 5]
  [ 6]
  [ 7]
  [ 8]
  [ 9]
  [10]]]
*************
[[10]
 [11]]
****************************************


In [60]:
a  = []
b = []
for i in range(1000):
    a.append(i)
    b.append(i)
a[10:17]

[10, 11, 12, 13, 14, 15, 16]

In [58]:
b[18]

18

In [14]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
class MyData(Dataset):
    def __init__(self, data_train, numstep, length):
        self.serial_number = data_train['serial_number'].value_counts()
        self.value = data_train.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.datalabel = data_train.loc[:, 'label'].values.tolist()
        self.input = []
        self.label = []

        for diskname in self.serial_number.index.sort_values():
            traindata_name = data_train[data_train.serial_number == diskname]
            self.datalabel = data_train.loc[:, 'label'].values.tolist()
            for i in range(len(traindata_name) - numstep):
                self.input.append(torch.Tensor(self.datas[i: i + numstep]))
                if(self.datalabel[i + numstep - 1] != 0):
                    self.label.append(torch.ones(1))
                else:
                    self.label.append(torch.zeros(1))
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]

In [17]:
data_train = MyData(data, 7, 6)
len(data_train)

num = 0
total = len(data_train)
sample_list = []

for X, y in data_train:
    num = num + y

    
len(num)

1